In [ ]:
%matplotlib inline

# Extract bike parking spaces from mongoDB, export as SVG for nesting, stitch nested SVGs 🚲 🚲 🚲

This notebook extracts geometries (areas, like polygons of parking spaces) from a mongoDB, then exports all areas in an svg file for nesting with SVGNest. One bin is used.

Created on:  2016-12-06  
Last update: 2016-12-06  
Contact: michael.szell@moovel.com, michael.szell@gmail.com (Michael Szell)

### Preliminaries

#### Parameters

In [ ]:
cityname = "amsterdam" # "amsterdam" "berlin"
mode = "bike" # do bike here. car is another file

#### Imports

In [ ]:
from __future__ import unicode_literals
import sys
import csv
import os
import math
from random import shuffle, choice, uniform
import random
import pprint
import requests
import gzip
from collections import defaultdict
import time
import datetime
import numpy as np
from numpy import *
from scipy import stats
import pyprind
import itertools
import logging
from ast import literal_eval as make_tuple
from collections import OrderedDict
from retrying import retry
from copy import deepcopy

import json
from xml.dom import minidom
from shapely.geometry import mapping, shape, LineString, LinearRing, Polygon, MultiPolygon
import shapely
import shapely.ops as ops
from shapely import affinity
from functools import partial
import pyproj
Projection = pyproj.Proj("+proj=merc +lon_0=0 +x_0=0 +y_0=0 +ellps=WGS84 +units=m +no_defs")
from scipy import spatial
from haversine import haversine
import overpass
apiop = overpass.API(timeout=600)
from scipy.ndimage.interpolation import rotate
from scipy.spatial import ConvexHull

import pymongo
from pymongo import MongoClient

# plotting stuff
import matplotlib.pyplot as plt

curtime = time.strftime("%Y%m%d_%H%M%S")
pp = pprint.PrettyPrinter(indent=4)

### Functions

In [ ]:
def coordinatesToSVGString(coo, xoffset = 0, yoffset = 0, idname = "", classname = "", rot = 0, centroidlatlon = [0,0]):
    svgstring = "\n  <polygon"
    if idname:
        svgstring += " id=\""+idname+"\""
    if classname:
        svgstring += " class=\""+classname+"\""
    svgstring += " points=\""
    strxylist = [str(coo[i][0]+xoffset)+","+str(coo[i][1]+yoffset) for i in range(coo.shape[0])]
    for s in strxylist:
        svgstring += s+" "
    svgstring += "\""
    svgstring += " moovel_rot=\""+str(rot)+"\"" # pseudo-namespace, because svgnest strips namespace info. http://stackoverflow.com/questions/15532371/do-svg-docs-support-custom-data-attributes
    centroid = [Polygon(coo).centroid.x, Polygon(coo).centroid.y]
    svgstring += " moovel_centroid=\""+str(centroid[0]+xoffset)+","+str(centroid[1]+yoffset)+"\""
    svgstring += " moovel_centroidlatlon=\""+str(centroidlatlon[0])+","+str(centroidlatlon[1])+"\""
    svgstring += "/>"
    return svgstring

def drawPolygon(poly, title=""): # poly is a shapely Polygon
    x, y = poly.exterior.xy
    fig = plt.figure(figsize=(4,4), dpi=90)
    ax = fig.add_subplot(111)
    ax.set_title(title)
    ax.plot(x, y)
    
def getLargestSubPolygon(multipoly): # multipoly is a shapely polygon or multipolygon
    # if its a polygon, do nothing, else give largest subpolygon
    if not (isinstance(multipoly, shapely.geometry.multipolygon.MultiPolygon)):
        return multipoly
    else:
        a = 0
        j = 0
        for i in range(len(multipoly)):
            if multipoly[i].area > a:
                j = i
                a = multipoly[i].area
        return multipoly[j]
    
def getSmallestSubPolygon(multipoly): # multipoly is a shapely polygon or multipolygon
    # if its a polygon, do nothing, else give largest subpolygon
    if not (isinstance(multipoly, shapely.geometry.multipolygon.MultiPolygon)):
        return multipoly
    else:
        a = float("inf")
        j = 0
        for i in range(len(multipoly)):
            if multipoly[i].area < a:
                j = i
                a = multipoly[i].area
        return multipoly[j]

def getTwoLargestSubPolygons(multipoly): # multipoly is a shapely polygon or multipolygon
    # if its a polygon, do nothing, else give two largest subpolygon
    if not (isinstance(multipoly, shapely.geometry.multipolygon.MultiPolygon)):
        return multipoly
    else:
        a = [multipoly[i].area for i in range(len(multipoly))]
        sortorder = sorted(range(len(a)), key=lambda k: a[k], reverse=True) # http://stackoverflow.com/questions/7851077/how-to-return-index-of-a-sorted-list
        return MultiPolygon([ multipoly[i] for i in sortorder[0:2] ])
    
def rotationToSmallestWidth(poly, rotdelta = 5): # poly is a shapely polygon
    # unit: degrees
    # decrease rotdelta to make it more precise (but also slower)
    rot = 0
    w = float("inf")
    
    for theta in range(0, 180, rotdelta):
        temp = affinity.rotate(poly, theta, origin='centroid')
        x, y = temp.exterior.coords.xy
        temp = np.array([[x[i],y[i]] for i in range(len(x))])
        objectwidth = max(temp[:, 0])-min(temp[:, 0])
        if objectwidth < w:
            w = objectwidth
            rot = theta
    return rot

def rotationToSmallestWidthRecursive(poly, maxdepth = 3, w = float("inf"), rot = 0, rotdelta = 10, depth = 1): # poly is a shapely polygon
    # unit: degrees
    # returns the angle the polygon needs to be rotated to be at minimum width
    # Note: Is not guaranteed to converge to the global minimum
    # Requires import numpy as np, from shapely import affinity
    if depth <= maxdepth:
        for theta in np.arange(rot-rotdelta*9, rot+rotdelta*9, rotdelta):
            temp = affinity.rotate(poly, theta, origin='centroid')
            x, y = temp.exterior.coords.xy
            temp = np.array([[x[i],y[i]] for i in range(len(x))])
            objectwidth = max(temp[:, 0])-min(temp[:, 0])
            if objectwidth < w:
                w = objectwidth
                rot = theta
        return rotationToSmallestWidthRecursive(poly, maxdepth, w, rot, rotdelta/10, depth+1)
    else:
        return rot
    
def getCoordinatesFromSVG(filepath, reversexdir = False, b = 1): # The SVG needs to have polygons with ids, embedded in gs
    doc = minidom.parse(filepath)  # parseString also exists
    path_strings = [path.getAttribute('points') for path
                    in doc.getElementsByTagName('polygon')]
    id_strings = [path.getAttribute('id') for path
                    in doc.getElementsByTagName('polygon')]
    class_strings = [path.getAttribute('class') for path
                    in doc.getElementsByTagName('polygon')]
    g_strings = [path.getAttribute('transform') for path
                in doc.getElementsByTagName('g')]
    rot_strings = [path.getAttribute('moovel_rot') for path
                    in doc.getElementsByTagName('polygon')]
    centroidlatlon_strings = [path.getAttribute('moovel_centroidlatlon') for path
                    in doc.getElementsByTagName('polygon')]
    doc.unlink()
    data = dict()
    numbins = 0
    for i,path in enumerate(path_strings):
        if class_strings[i] == "bin":
            numbins += 1
        if numbins == b:
            path = path.split()
            coo = []
            for temp in path:
                p = temp.split(",")
                try:
                    trans = g_strings[i] # looks like this: "translate(484.1119359029915 -1573.8819930603422) rotate(0)"
                    trans = trans.split()
                    trans = [float(trans[0][10:]), float(trans[1][0:-1])] # gives [484.1119359029915,-1573.8819930603422]
                except:
                    trans = [0,0]
                if reversexdir:
                    coo.append([-(float(p[0])+trans[0]), float(p[1])+trans[1]])
                else:
                    coo.append([float(p[0])+trans[0], float(p[1])+trans[1]])
            data[id_strings[i]] = dict()
            data[id_strings[i]]["coordinates"] = coo
            data[id_strings[i]]["rot"] = rot_strings[i]
            data[id_strings[i]]["class"] = class_strings[i]
            data[id_strings[i]]["centroidlatlon"] = centroidlatlon_strings[i].split(",")
            
        elif numbins > b:
            break
    return data

In [ ]:
# mongo connection
client = MongoClient()
db = client[cityname+'_derived']
ways = db['ways']
cursor = ways.find({"$and": [{"properties.amenity": "bicycle_parking"}, {"geometry.type": "Polygon"}, {"properties_derived.area": { "$gte": 1 }}]}).sort("properties_derived.area",-1)
numparkingareas = cursor.count()
print("There are " + str(numparkingareas) + " " + mode + " parking spaces in " + cityname)

## Get parking spaces for multiple SVG bins

### Features
1. add buffer around big tiles ✔️
2. give more bin space for small tiles on top of bins ✔️
3. pre-select tiles from DB and exhaust them fully (and uniquely!) ✔️
4. do not order big tiles by size, but more randomly ✔️
5. add id as id ✔️
6. prevent shadows

In [ ]:
pathdatain = '/Users/szellmi/Documents/lab-mobviz/_playground/parking-nesting/output/'+cityname+mode+'in/'
pathdataout = '/Users/szellmi/Documents/lab-mobviz/_playground/parking-nesting/output/'+cityname+mode+'out/'
# to find out maxbins, make 10 bins or so. If all have same file size, increase until file size gets small. Select maxbins to cover all big parts.
# to find out maxbigparts, set to around maxbins*4. Possibly decrease by looking at bigparts.svg 

In [ ]:
# get parking spaces (ALL in a city)
cursor = ways.find({"$and": [{"properties.amenity": "bicycle_parking"}, {"geometry.type": "Polygon"}, {"properties_derived.area": { "$gte": 1 }}]}).sort("properties_derived.area",-1)
random.seed(1)
scale = 0.6
erectparts = True
randomrotateparts = False
smallvsmedium = 11
buffereps = 5 # should be the same number as the distances between parts in SVGNest
height = 1200
width = 600
draw = False # for debugging purposes (drawing all big parts and bins) set this to True
eps = 0.000001

# pre-select all parts
idsused = set()
idsnotused = set()
indicesused = set()
alltiles = []
alltileskeys = []
alltilesarea = 0
for i,way in enumerate(cursor):
    npway = np.asarray(way["geometry"]["coordinates"])
    centroidlatlon = [Polygon(npway).centroid.x, Polygon(npway).centroid.y]
    npwayxy = [Projection(npway[i][0], npway[i][1]) for i in range(npway.shape[0])]
    npwayxy = np.asarray([[npwayxy[i][0],-npwayxy[i][1]] for i in range(npway.shape[0])])
    if erectparts:
        rot = 90+rotationToSmallestWidthRecursive(Polygon(npwayxy))
    elif randomrotateparts:
        rot = uniform(10, 350)
    else:
        rot = 0
    if rot:
        temp = affinity.rotate(Polygon(npwayxy), rot, origin='centroid', use_radians=False)
        x, y = temp.exterior.coords.xy
        npwayxy = np.array([[x[i],y[i]] for i in range(len(x))])
    objectwidth = max(npwayxy[:, 0])-min(npwayxy[:, 0])
    npwayxy[:, 0] -= min(npwayxy[:, 0])
    npwayxy[:, 1] -= min(npwayxy[:, 1])
    npwayxy *= scale
    objectwidth *= scale
    if objectwidth < width:
        objectheight = max(npwayxy[:, 1])
        idsnotused.add(way["_id"])
        coo = [[npwayxy[k][0], npwayxy[k][1]] for k in range(npwayxy.shape[0])]
        area = Polygon(coo).buffer(buffereps/2).area
        alltiles.append( { "_id": way["_id"], "width": objectwidth, "height": objectheight, "area": area, "coordinates": coo , "rot": rot, "centroidlatlon": centroidlatlon})
        alltileskeys.append(way["_id"])
        alltilesarea += area
    else:
        print("Object "+str(way["_id"])+" was too wide (" +str(objectwidth)+ " pixel) and was ignored.")


bigbin = Polygon([[0,0], [width,0], [width, height], [0, height]])
bigbinarea = bigbin.area
# change the big bin area according to the leftover tiles area
height = 1.25 * height * alltilesarea/bigbinarea
bigbin = Polygon([[0,0], [width,0], [width, height], [0, height]])
        
# Fill with parts
binarea = bigbin.area
binbound = np.array(bigbin.exterior.coords)
xpos = 0
ypos = height+1 # start placing the elements below the bin
yextent = 0
svg = "<svg xmlns=\"http://www.w3.org/2000/svg\" version=\"1.1\" width=\""+str(width)+"px\" height=\""+str(height)+"px\">"
svg += coordinatesToSVGString(binbound, 0, 0, str(0), "bin")
totalarea = 0

for j in range(len(idsnotused)):
    if len(idsnotused) == 0:
            break
    tile = alltiles[j]
    if tile["width"] <= width:
        if xpos + tile["width"] + 1 <= width: # there is space in this row
            xdelta = (xpos+1)
            ydelta = ypos
        else: # new row
            xdelta = 0
            ypos += yextent
            yextent = 0
            ydelta = ypos
            xpos = 0
        svg += coordinatesToSVGString(np.array([[tile["coordinates"][k][0], tile["coordinates"][k][1]] for k in range(np.array(tile["coordinates"]).shape[0])]), xdelta, ydelta, str(tile["_id"]), "tile", tile["rot"], tile["centroidlatlon"])
        yextent = max([yextent, tile["height"]])
        xpos += tile["width"]+1
        idsused.add(tile["_id"])
        idsnotused.remove(tile["_id"])
        totalarea += tile["area"]
    else:
        print("Object "+str(way["_id"])+" was too wide (" +str(max(npwayxy[:, 0]))+ " pixel) and could not be placed.")
svg += "\n</svg>"

with open(pathdatain + cityname + mode + "parking"+ str(0).zfill(3)  +"in.svg", "w") as f:
    f.write(svg)
                
print("First export done. " + str(len(idsnotused))+" tiles were not used.")

The result is all.svg in {{pathdataout}}.

## After SVGNest was executed, turn it into a nice SVG

In [ ]:
# read SVG
alltilesfinal = []
tiles = getCoordinatesFromSVG(pathdataout + str(0).zfill(3) + ".svg")
for key in tiles:
    if tiles[key]["class"] == "tile":
        npwayxy = np.array(tiles[key]["coordinates"])
        npwayxy = [[npwayxy[k,0], npwayxy[k,1]] for k in range(npwayxy.shape[0])]
        alltilesfinal.append({key: {"coordinates": npwayxy, "rot": tiles[key]["rot"], "centroidlatlon": tiles[key]["centroidlatlon"]}})

# Export
svg = "<svg xmlns=\"http://www.w3.org/2000/svg\" version=\"1.1\" width=\""+str(width)+"px\" height=\""+str(height)+"px\">"
for j, tile in enumerate(alltilesfinal):
    for i in tile:
        svg += coordinatesToSVGString(np.array([[tile[i]["coordinates"][k][0], tile[i]["coordinates"][k][1]] for k in range(np.array(tile[i]["coordinates"]).shape[0])]), 0, 0, i, "", tile[i]["rot"], tile[i]["centroidlatlon"])
svg += "\n</svg>"
with open(pathdataout + "all.svg", "w") as f:
    f.write(svg)